In [2]:
import math
from typing import Optional

### Theoretical estimates of nucleation size

Theoretical estimates of nucleation size considering the different solutions of Rice and Ruina (1983), Rubin and Ampuero (2005), and Chen and Lapusta (2009), compiled in Laspusta and Liu (2009)

Dynamic instability can only occur if the velocity weakening zone of the fault exceeds the nucleation size h*


Rice and Ruina, 1983

$$ h_{RR}^* = \frac{\pi}{4} \frac{\mu^* L}{(b - a) \sigma} $$ 

Rubin and Ampuero, 2005

$$ h_{RA}^* = \frac{2}{\pi} \frac{\mu^* b L}{(b - a)^2 \sigma} $$ 

Chen and Lapusta, 2009

$$ h^* = \left( \frac{\pi^2}{4} \right) h_{RA}^* $$ 

Where $\mu^*$ depends on crack mode as follows:

Mode III

$$\mu^* = \mu$$ 

Mode II

$$\mu^* = \frac{\mu}{1-\nu}$$ 

Variables

| Variable   | Description                                                      | Units         |
|------------|------------------------------------------------------------------|---------------|
| $h_{RR}^*$| Characteristic nucleation size (Rice and Ruina, 1983)        | m             |
| $h_{RA}^*$ | Characteristic nucleation size (Rubin and Ampuero, 2005)         | m           |
| $ h^* $    | Characteristic nucleation size (Chen and Lapusta, 2009)   | m             |
| $ \mu^* $   | Effective shear modulus                                        | Pa            |
| $ \mu $      | Shear modulus                                                  | Pa            |
| $ L $    | Characteristic length scale                                    | m             |
| $ b$        | Rate-and-state friction parameter                             | -             |
| $ a $      | Rate-and-state friction parameter                             | -             |
| $ \sigma $| Effective normal stress                                        | Pa            |
| $ \nu $      | Poisson's ratio (only relevant for Mode II)                    | -             |


Functions for each estimate

In [3]:
# kdc: add default key value pairs for each input

def mu_star(
    crack_mode: str, 
    shear_mod: float ,
    poisson_ratio: Optional[float] = None
) -> float:  
    '''Estimate mu_star based on crack mode'''
    
    if crack_mode == 'III':
        mu_star = shear_mod
    elif crack_mode == 'II':
        mu_star = shear_mod/(1-poisson_ratio)
    else:
        NameError("Crack mode is required to estimate mu_star") # EQs are mixed mode II and III so no mode I
    return mu_star

def nucleation_Rice_Ruina(mu_star,L,b,a,sigma):
    ''' Nucleation size under Rice and Ruina (1983) estimate based on linear stability analysis of steady sliding'''
    h_RR = (math.pi / 4) * (mu_star * L / ((b - a) * sigma))
    # L has units meters, sigma has units Pascals, mu_star has units Pascals
    return h_RR

def nucleation_Rubin_Ampuero(mu_star,b,L,a,sigma):
    '''Nucleation size under Rubin and Ampuero (2005) based on energy balance for quasi-static growing crack'''
    h_RA = (2 / math.pi) * (mu_star * b * L / ((b - a) ** 2 * sigma))
    # L has units meters, sigma has units Pascals, mu_star has units Pascals
    return h_RA

def nucleation_Chen_Lapusta(h_RA):
    '''Nucleation size under Chen and Lapusta (2005), 3D approximation'''
    h = (math.pi ** 2 / 4) * h_RA
    # h_RA has units meters
    return h

def nucleation_over_cell(nucleation_length,cell_size):
    '''Ratio of the nucleation length to the maximum grid size'''
    return nucleation_length/cell_size

### Estimate nucleation size for set of parameters

In [4]:
# mechanical parameters
a = 0.003
b = 0.006
shear_mod = 2.0e10 # Pa
L = 5e-3 # characteristic slip distance Dc, m
sigma = 20e6 # normal stress, Pa
poisson_ratio = 0.2

# maximum allowed grid size
grid_size_max = 300

# estimate mu_star as a function of crack mode
mu_star_II = mu_star('II',shear_mod,poisson_ratio)
mu_star_III = mu_star('III',shear_mod,poisson_ratio)

Estimate nucleation size

In [5]:
h_RR_modeII = nucleation_Rice_Ruina(mu_star_II,L,b,a,sigma)
h_RA_modeII = nucleation_Rubin_Ampuero(mu_star_II,b,L,a,sigma)
h_modeII = nucleation_Chen_Lapusta(h_RA_modeII)

h_RR_modeIII = nucleation_Rice_Ruina(mu_star_III,L,b,a,sigma)
h_RA_modeIII = nucleation_Rubin_Ampuero(mu_star_III,b,L,a,sigma)
h_modeIII = nucleation_Chen_Lapusta(h_RA_modeIII)

In [6]:
nucleation_sizes = {
    'h_RR_modeII': h_RR_modeII,
    'h_RA_modeII': h_RA_modeII,
    'h_modeII': h_modeII,
    'h_RR_modeIII':h_RR_modeIII,
    'h_RA_modeIII': h_RA_modeIII,
    'h_modeIII': h_modeIII
}

nucleation_sizes_values = list(nucleation_sizes.values())
min_nucleation_sizes = min(nucleation_sizes_values)
max_nucleation_sizes = max(nucleation_sizes_values)

print('Minimum nucleation size:',min_nucleation_sizes)
print('Maximum nucleation size:',max_nucleation_sizes)

Minimum nucleation size: 1308.996938995747
Maximum nucleation size: 6544.984694978736


Check ratio of nucleation size and minimum grid size

In [7]:
ratios = {
    'h_RR_modeII': nucleation_over_cell(h_RR_modeII, grid_size_max),
    'h_RA_modeII': nucleation_over_cell(h_RA_modeII, grid_size_max),
    'h_modeII': nucleation_over_cell(h_modeII, grid_size_max),
    'h_RR_modeIII': nucleation_over_cell(h_RR_modeIII, grid_size_max),
    'h_RA_modeIII': nucleation_over_cell(h_RA_modeIII, grid_size_max),
    'h_modeIII': nucleation_over_cell(h_modeIII, grid_size_max)
}

ratio_values = list(ratios.values())
min_ratio = min(ratio_values)
max_ratio = max(ratio_values)

print('Minimum ratio:',min_ratio)
print('Maximum ratio:',max_ratio)

if min_ratio < 3:
    print("Warning: your maximum grid size may be too large to properly resolve earthquake nucleation")

Minimum ratio: 4.363323129985823
Maximum ratio: 21.81661564992912
